In [6]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import re
import time

In [7]:
# Path to chromedriver
executable_path = {'executable_path': 'chromedriver'}
browser = Browser ('chrome', **executable_path, headless=False)

In [8]:
# Link to mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [9]:
# Convert the browser html to a soup object and then quit the broswer
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [10]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8875/testing-proves-its-worth-with-successful-mars-parachute-deployment/" target="_self">Testing Proves Its Worth With Successful Mars Parachute Deployment</a></div>

In [11]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.'

In [12]:
# Use the parent element to find the first a tag and save it as 'news_title'
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'Testing Proves Its Worth With Successful Mars Parachute Deployment'

In [13]:
# Retrieve space images
# URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [17]:
# Find and click the full image button
full_image_elem = browser.find_by_id('preload')
full_image_elem.click()

ElementDoesNotExist: no elements could be found with id "preload"

In [16]:
# Find and click the more info button
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

C:\Users\Brendan Dowd\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:504: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


ElementDoesNotExist: no elements could be found with link by partial text "more info"

In [48]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [49]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA17843_hires.jpg'

In [50]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17843_hires.jpg'

In [57]:
# Mars Weather
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(5)

In [58]:
html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

In [59]:
# 1. Find a tweet with the data-name 'Mars Weather'
mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [63]:
# 2. Search within the tweet for the p tag or span tag containing the tweet text. As Twitter is frequently making changes 
# the try/except will identify the tweet text using a regular expression pattern that includes the string 'sol' if there is
# no p tag with a class of 'tweet-text'

# Mars Weather Twitter Account URL

try:
    mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()
    mars_weather

except AttributeError:

    pattern = re.compile(r'sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    mars_weather

mars_weather

AttributeError: 'NoneType' object has no attribute 'text'

In [64]:
# Mars Hemispheres
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [66]:
hemisphere_image_urls = []

# 1: Get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

# 2: Loop through those links, click the link, find the sample anchor, and return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Next, find the sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere objects to list
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate backwards
    browser.back()

In [67]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [68]:
# Mars Facts
import pandas as pd
df = pd.read_html('https://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [69]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [70]:
browser.quit()